Importing Required Libraries

In [58]:
import numpy as np
import pandas as pd 
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [59]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('https://raw.githubusercontent.com/kushpatel19/Movie-Recommendation-Tool/main/movies.csv')

In [60]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [61]:
# number of rows and columns in the data frame
movies_data.shape

(4803, 24)

In [62]:
movies_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [63]:
movies_data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [90]:
# selecting the relevant features for recommendation
selected_features = ['genres','overview','keywords','tagline','title','cast','director']
print(selected_features)

['genres', 'overview', 'keywords', 'tagline', 'title', 'cast', 'director']


In [91]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [92]:
movies_data.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   0
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [93]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['overview']+movies_data['tagline']+' '+movies_data['title']+movies_data['cast']+' '+movies_data['director']

In [94]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [95]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

In [96]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [97]:
print(feature_vectors)

  (0, 5025)	0.13371253924161572
  (0, 16519)	0.08546429561652912
  (0, 26750)	0.14716077081021256
  (0, 20650)	0.12387999864214101
  (0, 18083)	0.16825915533486507
  (0, 29875)	0.1158084233164006
  (0, 34233)	0.14791793695571684
  (0, 28641)	0.1700826529966078
  (0, 27269)	0.1649734874756098
  (0, 35353)	0.14950596910619288
  (0, 34885)	0.17913087354881055
  (0, 2445)	0.21614013903528875
  (0, 22534)	0.03076638686057151
  (0, 34846)	0.07120459771960364
  (0, 10532)	0.1400820777736122
  (0, 6130)	0.16207532302891758
  (0, 1245)	0.120128362807027
  (0, 1513)	0.051597424454319654
  (0, 24926)	0.1649734874756098
  (0, 1555)	0.031197790299328267
  (0, 22808)	0.15828229785157244
  (0, 12021)	0.13597155195682187
  (0, 3488)	0.09783196022842411
  (0, 31824)	0.14242727606959252
  (0, 3159)	0.09618123770478947
  :	:
  (4802, 22348)	0.0607823440786599
  (4802, 13882)	0.07785039892643311
  (4802, 34311)	0.07691931803802896
  (4802, 10829)	0.07014169488358268
  (4802, 22610)	0.05703111121586624
  (

Cosine Similarity                                                           


---
cosine similarity is a measure of similarity between two sequences of numbers.  
It measures the similarity between vector lists by calculating the cosine angle between the two vector lists.


In [98]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [99]:
print(similarity)

[[1.         0.05371499 0.02093887 ... 0.02477005 0.01144605 0.00690098]
 [0.05371499 1.         0.0422509  ... 0.05457727 0.02916524 0.01607188]
 [0.02093887 0.0422509  1.         ... 0.02394685 0.02687246 0.01322031]
 ...
 [0.02477005 0.05457727 0.02394685 ... 1.         0.03054578 0.03502326]
 [0.01144605 0.02916524 0.02687246 ... 0.03054578 1.         0.03004636]
 [0.00690098 0.01607188 0.01322031 ... 0.03502326 0.03004636 1.        ]]


In [100]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from the user

In [112]:
# getting the movie name from the user
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : science


In [113]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

difflib :-  In-built module in the Python programming language consisting of different simple functions and classes that allow users to compare data sets

In [114]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Transcendence']


In [115]:
close_match = find_close_match[0]
print(close_match)

Transcendence


In [116]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

333


In [117]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.02044200292106145), (1, 0.046562474169744114), (2, 0.025305412249278552), (3, 0.011956783784820842), (4, 0.02259751951823834), (5, 0.005447043372901424), (6, 0.02182915888879738), (7, 0.02501159925977726), (8, 0.011189675844089132), (9, 0.02051069013187331), (10, 0.04145885247374622), (11, 0.036917234384350926), (12, 0.02408041326143729), (13, 0.025310012713806063), (14, 0.05755532430868442), (15, 0.028281614919921136), (16, 0.028128848649292933), (17, 0.02761670145551244), (18, 0.031937679977716), (19, 0.029967463714807047), (20, 0.03530512352216073), (21, 0.007764899962153455), (22, 0.02855563651784965), (23, 0.012328613196808912), (24, 0.03185011593411032), (25, 0.013395560290501091), (26, 0.02519433278067986), (27, 0.03415216648260209), (28, 0.02374684411608211), (29, 0.03327698026350894), (30, 0.014962407892442714), (31, 0.0564090916151535), (32, 0.023542755732954772), (33, 0.018486202848275464), (34, 0.007261027131058582), (35, 0.015760381584167496), (36, 0.066371638936082

In [118]:
len(similarity_score)

4803

In [119]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(333, 1.0), (1281, 0.2318877294082437), (2484, 0.1764372922226665), (2966, 0.15355342592718282), (634, 0.1466075155790534), (2817, 0.14626469729079042), (606, 0.14525271810074364), (39, 0.13111758760323355), (1625, 0.12510440658893904), (2026, 0.1227073992073672), (1997, 0.12164983939310747), (2752, 0.11871021337459305), (1153, 0.11822302637046216), (2636, 0.11607874505510496), (929, 0.11259929417512016), (447, 0.10899974351179163), (1661, 0.10859595028275464), (2354, 0.10634817766866365), (1517, 0.10025247177119609), (123, 0.09997850771248493), (1270, 0.09839934628124808), (1296, 0.09833215307831825), (607, 0.0970747969884079), (1279, 0.09598923796864392), (76, 0.09504091743298282), (3624, 0.09365721007863546), (4032, 0.09302564111451242), (1182, 0.0903780130062034), (1352, 0.08957352957142736), (695, 0.08915466436929276), (259, 0.08912753483320954), (242, 0.08636069851073325), (4336, 0.08602892232396019), (531, 0.08520532909054193), (95, 0.08346018807553762), (2824, 0.08227002915903

In [120]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<21):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Transcendence
2 . Hackers
3 . The Thirteenth Floor
4 . 2001: A Space Odyssey
5 . The Matrix
6 . WarGames
7 . Blackhat
8 . TRON: Legacy
9 . Virtuosity
10 . The Net
11 . Her
12 . Ex Machina
13 . Lucy
14 . Listening
15 . Outbreak
16 . Eagle Eye
17 . Antitrust
18 . Steve Jobs
19 . Repo Men
20 . The Matrix Revolutions


Movie Recommendation Sytem

In [121]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : zombie
Movies suggested for you : 

1 . Zombieland
2 . Kick-Ass 2
3 . Land of the Dead
4 . Emma
5 . The Joneses
6 . Night of the Living Dead
7 . Zombie Hunter
8 . Atonement
9 . Day of the Dead
10 . My Sister's Keeper
11 . ZMD: Zombies of Mass Destruction
12 . C.H.U.D.
13 . Gangster Squad
14 . 9
15 . Maggie
16 . Your Sister's Sister
17 . Ramona and Beezus
18 . Cursed
19 . Blue Jasmine
20 . Little Miss Sunshine
21 . I Am Legend
22 . Ride Along
23 . Howards End
24 . The Book of Eli
25 . The Helpers
26 . Tango & Cash
27 . Kick-Ass
28 . Damnation Alley
29 . Marvin's Room
